In [12]:
from flask import Flask
app = Flask(__name__)
import os
import re
import sqlite3
import csv
from flask import Flask
from flask import url_for, render_template, request, redirect

In [14]:
def cleanfiles():
    filelist = os.listdir()
    for item in filelist:
        if os.path.isfile(item) and ".html" in item and not "_p" in item:
            with open(item, encoding="utf-8") as f:
                text = f.read()
                notagstext = re.sub("<.*?>", "", text, flags=re.DOTALL)
                puretext = re.sub("{.*?}", "", notagstext, flags=re.DOTALL)
            newfile = item + "_p"
            with open(newfile, "w", encoding="utf-8") as n:
              n.write(puretext)
            filelist.append(newfile)
            with open(newfile, encoding="utf-8") as nf:
                lines = nf.readlines()
                senselines = []
                for line in lines:
                    if re.search('[а-яА-ЯёЁ]', line):
                        if "&nbsp;" in line:
                            lline = re.sub("&nbsp;", " ", line)
                        else:
                            lline = line
                        senselines.append(lline)
            plainfile = newfile + "l"
            with open(plainfile, "w", encoding="utf-8") as pl:
                for ruline in senselines:
                    pl.write(ruline)
            with open(plainfile, encoding="utf-8") as p:
                pp = p.read()
                pp_cl = re.sub("[A-Za-z0-9.-_-#]", "", pp)
                pp_cle = re.sub("\n", " ", pp_cl)
                pp_clean = re.sub("<!--", ";", pp_cle)
            with open(plainfile, "w", encoding="utf-8") as s:
                s.write(pp_clean)
            filelist.append(plainfile)
    return filelist

In [16]:
def makeref():
    filelist = cleanfiles()
    for element in filelist:
        if ".html" in element and not "_" in element:
            with open(element, encoding="utf-8") as e:
                lines = e.readlines()
                ref = ";" + lines[1]
            transmit = element + "_pl"
            with open(transmit, "a", encoding="utf-8") as t:
               t.write(ref)

In [18]:
def maketable():
    filebase = []
    filelist = cleanfiles()
    makeref()
    for element in filelist:
        if os.path.isfile(element) and ".html_pl" in element:
            filebase.append(element)
    csvfile = "paper.csv"
    with open (csvfile, "w", encoding="utf-8") as c:
        c.write("name,content,ref\n")
        for file in filebase:
            with open(file, encoding="utf-8") as f:
                text = f.read()
                parts = text.split("<")
                referplace = parts[-1]
                refer = re.search("http://(.+)", referplace).group()
                c.write(file + ";" + text + ";" + refer + "\n")
    return csvfile

In [20]:
def makebase():
    table = maketable()
    with open (table, encoding="utf-8") as f:
        filereader = csv.reader(f, delimiter=';', quotechar='|')
        fileread = list(filereader)
        rowlist = fileread[1::]
    conn = sqlite3.connect('paper.db', timeout=10)
    c = conn.cursor()
    c.execute("DROP TABLE if exists paper")
    c.execute("CREATE TABLE if not exists paper(filename text, filetext text, refer text)")
    for row in rowlist:
        c.execute("INSERT INTO paper (filename, filetext, refer) VALUES (?, ?, ?)", (row[0], row[1], row[-1]))
        conn.commit()

In [37]:
@app.route('/')
def index():
    return render_template("index.html")
@app.route('/results')
def results():
    if request.args:
        search = request.args["search"]
        conn = sqlite3.connect('paper.db', timeout=10)
        c = conn.cursor()
        c.execute("SELECT * FROM paper WHERE filetext like '%?%' ", search)
        printout = c.fetchall()
        return render_template("results.html", search=search)
    return redirect(url_for('index'))
def main():
    makebase()
    app.run(debug=False)
main()

AssertionError: View function mapping is overwriting an existing endpoint function: index